**Extracción y captura de información**

In [12]:
import requests
import pandas as pd

##Creación de archivo/dataframe total, juntando 2022 y 2023

# URL de la API de KoboToolbox
url = "https://kc.kobotoolbox.org/api/v1/data/1239161?format=json "

# Token de acceso
token = "6c5380b2230830a0e9e89868e064a70447adad25"

# ID del formulario que deseas obtener
form_id = "aJS64aAnMVsGQZ6tTPo9QP"

# Parámetros de la solicitud GET
params = {
    "format": "json",
    "limit": 2000,
    "id": form_id
}

# Encabezados de la solicitud con el token de acceso
headers = {
    "Authorization": f"Token {token}",
}

# Envía la solicitud GET
response = requests.get(url, params=params, headers=headers)
if response.status_code == 200:

    # Convierte los datos de la API en un DataFrame

    data = response.json()
    df = pd.DataFrame(data)
    Campos_interes = [
    'Fecha',
    'Nombre_del_monitor',
    'declaratoria',
    'comunidad',
    'Registre_la_ubicacion_en_donde',
    'Estoy_de_acuerdo',
    'group_ENCUESTA/Genero',
    'group_ENCUESTA/lugar_origen',
    'group_ENCUESTA/Tiempo_viviendo_comunidad',
    'group_ENCUESTA/edad_encuestado',
    'group_ENCUESTA/nivel_educativo_encuestado',
    'group_ENCUESTA/actividad_economica_fliar',
    'group_ENCUESTA/group_compo_hogar/N_personas_hogar',
    'group_ENCUESTA/group_compo_hogar/mas_5_personas',
    'group_ENCUESTA/group_compo_hogar/N_personas_dependen',
    'group_ENCUESTA/group_compo_hogar/_Cu_les_son_las_dos_cas_de_TODO_EL_HOGAR/pimera_opcion',
    'group_ENCUESTA/group_compo_hogar/_Cu_les_son_las_dos_cas_de_TODO_EL_HOGAR/segunda_opcion',
    'group_ENCUESTA/act_economica_ppal_encuestado',
    'group_ENCUESTA/obj_actividad_econ_ppal',
    'group_ENCUESTA/gustaria_otra_actividad',
    'group_ENCUESTA/Por_que_a_la_respuesta',
    'group_ENCUESTA/actividades_gustaria',
    'group_ENCUESTA/otra_actividad_gustaria',
    'group_ENCUESTA/ha_tenido_credito',
    'group_ENCUESTA/significado_vivir_bien',
    'group_ENCUESTA/Cambiaria_lugar_donde_vive',
    'group_ENCUESTA/Tiene_hijos',
    'group_ENCUESTA/asociaciones_grupos_colectivos',
    'group_ENCUESTA/Pertenece_a_esa_asociacion',
    'group_ENCUESTA/group_social_salud/subsidio',
    'group_ENCUESTA/group_social_salud/salud',
    'group_ENCUESTA/group_social_salud/pension',
  ]

    df_api = df[Campos_interes]

    # Cargar datos del 2022 desde el archivo Excel
    df_excel = pd.read_excel('ACADESAN_2022.xlsx')


    # Generación del archivo final juntando la info del API y el excel
    df_final = pd.concat([df_api, df_excel], axis=0)


    # Modificación del nombre de las cabeceras
    df_final.columns = [
                          'fecha' ,
                          'monitor',
                          'declaratoria',
                          'comunidad',
                          'ubicación',
                          'aprueba_encuesta',
                          'genero',
                          'lugar_origen',
                          'tiempo_residencia_comunidad',
                          'edad',
                          'nivel_educativo',
                          'actividad_economica_familiar',
                          'no_personas_hogar',
                          'no_personas_hogar_M5',
                          'no_personas_dependiente',
                          'primera_actividad_economia_hogar',
                          'segunda_actividad_economia_hogar',
                          'actividad_economica_encuestado',
                          'objetivo_actividad_economica',
                          'gustaria_otra_actividad',
                          'por_que_quiere_otra_actividad',
                          'actividad_gustaria',
                          'otra_actividad_gustaria',
                          'ha_tenido_credito',
                          'significado_vivir_bien',
                          'cambiaria_lugar_donde_vive',
                          'tiene_hijos',
                          'asociaciones_grupos_colectivos',
                          'pertenece_asociacion',
                          'servicio_salud',
                          'tipo_vinculacion',
                          'tipo_pension',
                          ]

else:
    print("Error al obtener los datos:", response.status_code)



**Transformación de datos**

In [ ]:
# Cambiar formato a la columna fecha
df_final['fecha'] = pd.to_datetime(df_final['fecha'], format='%Y-%m-%d')

# Obtener solo año, mes y día sin horas y segundos
df_final['fecha'] = df_final['fecha'].dt.date

#Crear la columna de numero de integrantes familiares
df_final["numero_integrantes_familiares"] = ''

# Crear una nueva columna 'Numero_Integrantes_Familiares' y asignarle los valores de 'no_personas_hogar'
df_final['numero_integrantes_familiares'] = df_final['no_personas_hogar']

# Actualizar solo los valores que cumplen la condición
df_final['numero_integrantes_familiares'] = df_final['numero_integrantes_familiares'].mask(df_final['no_personas_hogar'] == "mas_de_5", "5")


# Hasta el momento creamos la columna y le asignamos el valor 5 ahora cambiamos los formatos

# Asignamos los formatos a las columnas
df_final['fecha'] = pd.to_datetime(df_final['fecha'], errors='coerce')
df_final.fillna(0, inplace=True)


df_final['no_personas_hogar_M5'] = df_final['no_personas_hogar_M5'].astype(int)
df_final['tiempo_residencia_comunidad'] = df_final['tiempo_residencia_comunidad'].astype(float)
df_final['numero_integrantes_familiares'] = df_final['numero_integrantes_familiares'].astype(int)

df_final.info()


df_final["numero_integrantes_familiares"] = df_final["numero_integrantes_familiares"] + df_final["no_personas_hogar_M5"]

# Remplazar valores atípicos de la columna llamada numero_integrantes_familiares

mapeo = {57: 5, 107: 10, 67: 6, 88: 8}

df_final['numero_integrantes_familiares'] = df_final['numero_integrantes_familiares'].replace(mapeo)

df_final['genero'] = df_final['genero'].replace(0, "No responde")

df_final['primera_actividad_economia_hogar'] = df_final['primera_actividad_economia_hogar'].replace(0, "No responde")

df_final.to_excel('archivo_final.xlsx', index=False)




**Preanálisis y visualización de datos**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Mostrar las primeras filas del DataFrame
print(df_final.info())

# Resumen estadístico de los datos
print(df_final.describe())

# Determinar qué columnas son categóricas y cuáles son numéricas
columnas_numericas = df_final.select_dtypes(include=['int64', 'float64']).columns
columnas_categoricas = df_final.select_dtypes(include=['object']).columns

# Mostrar qué columnas son categóricas y cuáles son numéricas
print("Columnas numéricas son: ", columnas_numericas)
print("Columnas categóricas son:", columnas_categoricas)

# Visualización básica de los datos
# Por ejemplo, un histograma para evaluar el número de personas integrantes en el hogar
plt.figure(figsize=(30, 6))
sns.histplot(df_final['numero_integrantes_familiares'], bins=30, kde=False, color='red')
plt.title('Número de personas integrantes en el hogar')
plt.xlabel('Cantidad de personas')
plt.ylabel('Frecuencia')
plt.show()

# Visualización básica de los datos
# Por ejemplo, un histograma del tiempo de residencia en la comunidad
plt.figure(figsize=(30, 6))
sns.histplot(df_final['tiempo_residencia_comunidad'], bins=20, kde=True, color='green')
plt.title('Tiempo de residencia en la comunidad')
plt.xlabel('Cantidad de Años')
plt.ylabel('Frecuencia')
plt.show()

# Visualización básica de los datos
# Por ejemplo, cantidad de comunidades encuestadas
plt.figure(figsize=(15, 6))
sns.histplot(df_final['comunidad'], bins=40, kde=True, palette = 'viridis')
# Rotar las etiquetas del eje x
plt.xticks(rotation=45)  # Puedes ajustar el ángulo de rotación según tus necesidades
plt.title('Comunidades Encuestadas')
plt.xlabel('Comunidades')
plt.ylabel('Encuestas')
plt.show()


import seaborn as sns
import matplotlib.pyplot as plt

#### Gráfico de barra sobre la cantidad de personas encuestadas por género
conteo_genero = df_final['genero'].value_counts()
# Tamaño del gráfico (ancho x alto)
plt.figure(figsize=(10, 6))
# Dibujar las barras con las barras en el eje x
sns.barplot(x=conteo_genero.index, y=conteo_genero.values, palette='viridis')
# Agregar etiquetas y título
plt.xlabel('Género')
plt.ylabel('Cantidad de personas encuestadas')
plt.title('Encuestas realizadas por género')
# Mostrar el gráfico
plt.show()

#### Actividad económica principal en el hogar
# Contar la cantidad de valores en la columna 'genero'
conteo_actividad = df_final['primera_actividad_economia_hogar'].value_counts()
# Tamaño del gráfico (ancho x alto)
plt.figure(figsize=(10, 6))
# Dibujar las barras con las barras en el eje x
sns.barplot(x=conteo_actividad.values, y=conteo_actividad.index, palette='viridis')
# Agregar etiquetas y título
plt.xlabel('Cantidades')
plt.ylabel('Actividades económicas')
plt.title('Actividad económica principal en el hogar')
# Mostrar el gráfico
plt.show()

#Matriz de correlación entre variables numéricas:
# Filtrar solo las columnas numéricas para el análisis de correlación
columnas_numericas = df_final.select_dtypes(include=['int64', 'float64'])

# Calcular la matriz de correlación
matriz_correlacion = columnas_numericas.corr()

# Visualizar la matriz de correlación utilizando un mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matriz de Correlación')
plt.show()


Importar base de datos

In [ ]:
#importar base de datos

!pip install mysql-connector
!pip install pymysql

import mysql.connector
from sqlalchemy import create_engine
import pandas as pd

# Cargar los datos desde el excel
datos_excel = pd.read_excel('archivo_final.xlsx')

# Definición de variables
table = "Datos_Finales"
host = "sql10.freesqldatabase.com"
user = "sql10699955"
password = "g8LNzMrxda"
database = "sql10699955"

# Construir la URL de conexión
url = f"mysql+pymysql://{user}:{password}@{host}/{database}"

# Crear la conexión a la base de datos
engine = create_engine(url, echo=False)

# Escribir los datos en la tabla de la base de datos
datos_excel.to_sql(name=table, con=engine, if_exists='append', index=False)